# Appendix: Testing our Implementation 

This notebook walks through how to test our overall VI implementation through statistical simulations and problem simplifications. It is intended as a complement to automated unit tests, which test component-wide analysis.

In [9]:
# Imports 
%load_ext autoreload
%autoreload 2

import torch

import sys 
sys.path.append('..')
from src.statistical_model import SingleFactorCFA
from src.variational_family import SingleCFAVariationalFamily
from src.variational_sem import VIOptimisationParameters, SingleCFAVIModel
from src.analytical_variational_inference.analytical_variational_infernce import single_factor_cfa_mfvb
from src.analysis.sampling import create_sample_from_qvar, SingleCFAVariationalParameters
from src.mcmc.mcmc import single_factor_cfa_mcmc
from torch.distributions import MultivariateNormal as mvn


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Testing Framework 

In general, any implementation of the VI algorithm should be debugged in two steps: 

* **Predictable Input and Output** It produces the correct answer to a known input and a known output.
* **Convergence for Non-Latent Parameter Values** The algorithm has properly converged: both its loss functions and individual parameter values. 
* **Spot-check for eta parameter values** Eta parameter values have also converged 

In [10]:
# Set seed for reproducibility 
torch.manual_seed(42)

In [11]:
# Simulated Data

N = 300 
M = 3 
nu = torch.tensor([5.0, 6.0, 2.25])
sig2 = torch.tensor([0.565])
lam = torch.tensor([0.75, 1.05])
psi = torch.tensor([0.8, 1.10, 0.66])

#Generate Latent Parameter Values 
eta = torch.randn(N)*torch.sqrt(sig2)

#Concatenate lambda values 
lam1_fixed = torch.tensor([1.0])
lam_full = torch.cat((lam1_fixed, lam))

#Generate y values based on User Input
# yi ~ id Normal(nu + eta_i * lam, diag(psi)), yi /in R^m
#cov:
like_dist_cov = torch.diag(psi) #m*m tensor 
#means: want a n*m vector of means
like_dist_means = torch.matmul(eta.unsqueeze(1), lam_full.unsqueeze(0)) + nu

y_data_sim = mvn(like_dist_means, covariance_matrix= like_dist_cov).rsample() 

In [12]:
#Hyper-parameter values

#sig_2 ~ InvGamma
sig2_shape = torch.tensor([0.5])  
sig2_rate = torch.tensor([0.5])  

#psi ~ iid Inv Gamma for j = 1..m 
psi_shape = torch.tensor([0.5])  
psi_rate = torch.tensor([0.005])  

#nu ~ iid Normal for j = 1...m
nu_sig2 = torch.tensor([100.0])  
nu_mean = torch.tensor([0.0])

#lam_j | psi_j ~ id Normal(mu, sig2*psi_j)
lam_mean = torch.tensor([0.0])
lam_sig2 = torch.tensor([1.0])

hyper_params = {"sig2_shape": sig2_shape, "sig2_rate": sig2_rate, "psi_shape": psi_shape, "psi_rate": psi_rate, "nu_sig2": nu_sig2, "nu_mean": nu_mean, "lam_mean": lam_mean, "lam_sig2": lam_sig2}

# Part 1 - Single Component Tests

In [25]:
VI_OPTIMISATION_PARAMETERS = VIOptimisationParameters(num_iterations = 20_000, relative_error_threshold= 10e-4, patience = 100)

## Part 2: Estimate all variables 

In [26]:
degenerates = {}
model = SingleCFAVIModel(y_data = y_data_sim, hyper_params= hyper_params, degenerates = degenerates)

In [27]:
model.optimize(optimisation_parameters= VI_OPTIMISATION_PARAMETERS, filename = 'tensorboard_runs/simulated_VIModel_test', K = 10, alpha =1)

 39%|███▉      | 7766/20000 [04:09<06:33, 31.07it/s]

VI converged at step t =  7766


In [ ]:
relative_errors = [error.item() for error in model.results.convergence_data.relative_errors]

In [39]:
plot(relative_errors)

NameError: name 'plot' is not defined

In [29]:
%reload_ext tensorboard
%tensorboard --logdir tensorboard_runs/simulated_VIModel_test_K_10_alpha1/

Reusing TensorBoard on port 6006 (pid 4283), started 0:16:22 ago. (Use '!kill 4283' to kill it.)

## Estimate all parameters 